<a href="https://colab.research.google.com/github/KhalidAlHabbash/NanoGPT/blob/main/NanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-11-11 19:27:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  3.36MB/s    in 0.3s    

2025-11-11 19:27:33 (3.36 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [25]:
import torch
import tiktoken
import torch.nn as nn
from torch.nn import functional as F
batch_size = 64 
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [26]:
# read in input text
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [27]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [28]:
# Tiktoken o200k_base tokenizer
enc = tiktoken.get_encoding("o200k_base")
print(enc.n_vocab)

200019


In [29]:
# Tokenize entire input text
data = torch.tensor(enc.encode(text), dtype=torch.long)
print(data.shape)
print(data[:500])

torch.Size([297606])
tensor([  7127,  84479,    734,  13036,    581,  18988,   1062,   6544,     11,
          9598,    668,  10591,    364,   2594,    734, 116872,     11,  10591,
           364,   7127,  84479,    734,   3575,    553,    722,  33944,   7542,
           316,   1076,   1572,    316,   2079,   1109,   1715,   2594,    734,
         80773,     13,  33944,    364,   7127,  84479,    734,   7127,     11,
           481,   1761, 149492,    385,   3145, 137928,    382,  20915,  20935,
           316,    290,   1665,    364,   2594,    734,   2167,   1761,   1507,
            11,    581,   1761,   1507,    364,   7127,  84479,    734,  12845,
           765,  15874,   2395,     11,    326,  22782,    679,  33994,    540,
          1039,   2316,   3911,    558,   3031,   1507,    261,  75722,   1715,
          2594,    734,   3160,    945,  11695,    402,   1507,     26,   1632,
           480,    413,   4167,     25,   4194,     11,   4194,   1703,  17422,
         84479,    

In [30]:
#Split into training and validation sets (90/10)
train_percentage = int(0.9 * len(data))
train_set = data[:train_percentage]
val_set = data[train_percentage:]
print(len(train_set), len(val_set))

267845 29761


In [31]:
print(train_set[:context_size])

tensor([  7127,  84479,    734,  13036,    581,  18988,   1062,   6544,     11,
          9598,    668,  10591,    364,   2594,    734, 116872,     11,  10591,
           364,   7127,  84479,    734,   3575,    553,    722,  33944,   7542,
           316,   1076,   1572,    316,   2079,   1109,   1715,   2594,    734,
         80773,     13,  33944,    364,   7127,  84479,    734,   7127,     11,
           481,   1761, 149492,    385,   3145, 137928,    382,  20915,  20935,
           316,    290,   1665,    364,   2594,    734,   2167,   1761,   1507,
            11,    581,   1761,   1507,    364,   7127,  84479,    734,  12845,
           765,  15874,   2395,     11,    326,  22782,    679,  33994,    540,
          1039,   2316,   3911,    558,   3031,   1507,    261,  75722,   1715,
          2594,    734,   3160,    945,  11695,    402,   1507,     26,   1632,
           480,    413,   4167,     25,   4194,     11,   4194,   1703,  17422,
         84479,    734,   5045,   2195, 

In [32]:
def get_batch(type):
    data = train_set if type == "train" else val_set
    # Generate random ints to access data (no. of random ints = batch_size)
    indices = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in indices])
    # y is off by 1 so that we can generate multiple training examples from just one input
    y = torch.stack([data[i+1:i+context_size+1] for i in indices])
    x,y = x.to(device), y.to(device)
    return x,y

In [33]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # Preserve computation, no need to calculate gradients
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [34]:
class attentionHead(nn.Module):
    """ Single head of attention """

    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias=False) # Linear layer (x * W(Q,K,V) without the bias)
        self.key = nn.Linear(n_embd, head_size, bias=False) 
        self.value = nn.Linear(n_embd, head_size, bias=False) 
        self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size)))
        self.dropout = nn.Dropout(dropout) # add dropout to prevent overfitting and encourage more robustness


    def forward(self, x):
        B,T,C = x.shape

        k = self.key(x) # get key vector (B,T, head_size)
        q = self.query(x) # get query vector (B,T, head_size)

        # compute attention scores
        attentionScores = q @ k.transpose(-2,-1) * k.shape[-1] ** -0.5 # --> (B,T,T)
        attentionScores = attentionScores.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # Mask future tokens in current context
        attentionScores = F.softmax(attentionScores, dim=-1) # apply softmax across each row (represets one token query)
        attentionScores = self.dropout(attentionScores)

        v = self.value(x) # (B,T,head_size)
        out = attentionScores @ v # (B,T,T) @ (B,T,head_size) = (B,T,head_size)
        return out

In [35]:
class MultiAttentionHead(nn.Module):
    """ multiple heads of attention in parallel"""

    def __init__(self, head_size, num_heads):
        super().__init__()
        self.attentionHeads = nn.ModuleList([attentionHead(head_size) for _ in range(num_heads)]) 
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        concat = torch.cat([ah(x) for ah in self.attentionHeads], dim=-1) # concatenate all feature vectors across each attention head
        return self.dropout(self.proj(concat)) # mix information across concatenated feature vectors

In [36]:
class MultiLayerPerceptron(nn.Module):
    """ mlp layer of the neural network (linear layer followed by non-linear layer)""" 

    def __init__(self, n_embd):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.mlp(x)

In [37]:
class Block(nn.Module):
    """ one block in a layer (attention  + feed forward (mlp) ) """

    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd // n_heads
        self.multiAttentionHeads = MultiAttentionHead(head_size, n_heads)
        self.mlp = MultiLayerPerceptron(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)


    def forward(self, x):
        x = x + self.multiAttentionHeads(self.ln1(x)) # maintain residual connections
        x = x + self.mlp(self.ln2(x)) # add new result vector of the mlp layer back in to the original vector
        return x

In [ ]:
class GenerativePretrainedTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.word_embedding_table = nn.Embedding(enc.n_vocab, n_embd)
        self.positional_embedding_table = nn.Embedding(context_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_heads=n_head) for _ in range(n_layer)])
        self.final_ln = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, enc.n_vocab)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.word_embedding_table(idx) # (B,T,C)
        pos_emb = self.positional_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.final_ln(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = GenerativePretrainedTransformer()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

164.553555 M parameters


KeyboardInterrupt: 